# 高斯过程
**高斯过程**是一种通用的监督学习方法，旨在解决和概率分类问题

优点：
1. 预测内插函数：对于一个新的观测样本，如果它的特征在训练集中出现过，则称其为内插观察。
2. 预测是概率性的
3. 通用性：可以指定不同的内核

缺点：
1. 非稀疏，使用整个样本/特征信息来执行预测
2. 在高维空间中失去效率



## 高斯过程回归
作用GPR时需先指定高斯过程的先验（包括均值和协方差），先验均值通常假设为0（`normalize_y=False`）或训练集的均值（`normalize_y=True`），先验协方差是通过kernel函数指定

kernel函数的超参数通过最大化对数边缘似然函数`log-marginal-likehood,LML`的优化器来拟合，由于LML可能存在多个局部最优解，因此可以通过`n_restarts_optimizer`参数来多次启动优化器，优化第一次运行总是从初始参数开始，随后的运行根据允许值范围内随机选择的超参数进行，如果希望保持初始参数不变，可以将optimizer的参数设置为`None`

关于参数`alpha`：标量代表全局噪声水平，数组代表每个数据点的噪声水平.
**在高斯过程回归中，如果在目标值中存在一些噪声，那么将适度的噪声水平加入到核矩阵的对角线上，可以帮助处理在训练过程中可能出现的数值问题，这相当于对核矩阵进行了Tikhonov正则化。另一种方法是通过将 WhiteKernel 组件添加到核中来明确指定噪声水平，这个组件可以从数据中估计出全局噪声水平。总之，加入噪声水平可以提高模型的稳定性和泛化性能。**

## 关于GaussianProcessRegressor模块的功能
1. 可以在没有先验拟合的情况下进行预测，直接基于GP先验进行预测
2. 利用sample_y函数生成一些采样输出，绘制置信区间
```python
import numpy as np
from scipy.stats import norm

# assume X_test is a 2D array of test points
# with shape (n_samples, n_features)
y_mean, y_std = gp.predict(X_test, return_std=True)
n_samples = 1000
y_samples = gp.sample_y(X_test, n_samples)
z_lower = norm.ppf(0.025)
z_upper = norm.ppf(0.975)
lower = y_mean - y_std * z_lower
upper = y_mean + y_std * z_upper

```
3. 提供在外部实现`log_marginal_likehood`的方法，可以使用梯度优化方法（如共轭梯度，拟牛顿方法），也可以使用马尔笠夫链蒙特卡罗（MCMC）等其他方法实现
```python
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF
import numpy as np

# 创建一些样本数据
rng = np.random.RandomState(0)
X = rng.uniform(0, 5, 20)[:, np.newaxis]
y = np.sin((X[:, 0] - 2.5) ** 2)

# 初始化高斯过程回归器和 RBF 核
kernel = RBF()
gp = GaussianProcessRegressor(kernel=kernel, alpha=0.1)

# 使用 Marginal Likelihood 选择最佳的核超参数
theta = gp.log_marginal_likelihood(X, y)
print('Best kernel parameters:', np.exp(theta))

```